In [1]:
import sys

import sklearn

import numpy as np
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix

In [2]:
#LOADING DATASET

train = pd.read_csv('/Users/prachisadarangani/Documents/Syracuse University /fALL 2023 SEM 3/AML/Assignment 6/digit-train.csv')
test = pd.read_csv('/Users/prachisadarangani/Documents/Syracuse University /fALL 2023 SEM 3/AML/Assignment 6/digit-test.csv')

In [3]:
train

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4193,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4194,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4195,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#Dividing it into X and y

X_train = train.drop('label', axis=1)
y_train = train['label']

X_test = test.drop('label', axis=1)
y_test = test['label']

In [5]:
#decision tree

model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_train,y_train)

DecisionTreeClassifier(random_state=42)

In [6]:
#prediction
y_pred = model_dt.predict(X_test)

In [7]:
#checking accuracy and precision
accuracy = accuracy_score(y_pred,y_test)
precision = precision_score(y_pred,y_test, average = 'weighted')
print("accuracy:", accuracy)
print("precision", precision)

accuracy: 0.7832301095759886
precision 0.7844379209983415


In [8]:
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.88      0.87       414
           1       0.90      0.94      0.92       478
           2       0.74      0.76      0.75       420
           3       0.72      0.72      0.72       446
           4       0.79      0.79      0.79       404
           5       0.70      0.71      0.71       388
           6       0.79      0.78      0.78       404
           7       0.85      0.84      0.85       465
           8       0.67      0.65      0.66       393
           9       0.75      0.73      0.74       386

    accuracy                           0.78      4198
   macro avg       0.78      0.78      0.78      4198
weighted avg       0.78      0.78      0.78      4198



In [29]:
#checking cross validation score 
scores = cross_val_score(model_dt, X_train, y_train, cv=3, scoring='accuracy')
print("cross-validation scores:", scores)

cross-validation scores: [0.72857143 0.73766976 0.74839171]


In [10]:
#fine-tuning
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_dist = { 
    'criterion': ['gini', 'entropy'],
    'max_depth': np.arange(10, 200, step=2), 
    'min_samples_split': np.arange(2, 26), 
    'min_samples_leaf': np.arange(1, 26) 
}

random_search = RandomizedSearchCV(model_dt, param_distributions=param_dist,
                                   n_iter=100, cv=3, n_jobs=-1, verbose=2, random_state=42)

# Perform the randomized search
random_search.fit(X_train, y_train)

# Print the best hyperparameters and corresponding score
print("Best Hyperparameters: ", random_search.best_params_)

best_model = random_search.best_estimator_

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best Hyperparameters:  {'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 118, 'criterion': 'entropy'}


In [11]:
#prediction and checking accuracy and precision
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_pred,y_test)
precision = precision_score(y_pred,y_test, average = 'weighted')
precision
print("accuracy:", accuracy)
print("precision", precision)


accuracy: 0.7908527870414483
precision 0.7917953083026465


In [12]:
#checking cross-validation score
scores = cross_val_score(best_model, X_train, y_train, cv=3, scoring='accuracy')
print("cross validation scores", scores)

cross validation scores [0.73285714 0.76054325 0.7498213 ]


In [13]:
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.90      0.88       414
           1       0.92      0.93      0.92       478
           2       0.73      0.79      0.76       420
           3       0.76      0.74      0.75       446
           4       0.78      0.80      0.79       404
           5       0.66      0.68      0.67       388
           6       0.83      0.81      0.82       404
           7       0.84      0.80      0.82       465
           8       0.71      0.68      0.70       393
           9       0.77      0.74      0.75       386

    accuracy                           0.79      4198
   macro avg       0.79      0.79      0.79      4198
weighted avg       0.79      0.79      0.79      4198



In [14]:
#confusion matrix
confusion_dt = confusion_matrix(y_test,y_pred)
confusion_dt

array([[372,   0,   8,   2,   3,  13,   9,   0,   4,   3],
       [  0, 444,   7,   2,   3,   4,   3,   6,   7,   2],
       [  6,   9, 330,   7,   6,   5,  12,  15,  22,   8],
       [ 13,   4,  21, 329,  11,  35,   3,   9,  10,  11],
       [  1,   2,   9,   3, 325,  13,   5,  13,   8,  25],
       [ 15,   3,  12,  46,   8, 265,   9,   3,  19,   8],
       [  6,   7,  17,   5,  13,  13, 328,   3,  11,   1],
       [  9,   3,   9,  10,  17,  17,   0, 374,   5,  21],
       [  3,   9,  34,  17,   9,  20,  23,   2, 268,   8],
       [  6,   1,   2,  13,  23,  14,   2,  18,  22, 285]])

In [15]:
#let's build the second model 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

model_nb = MultinomialNB()
model_nb.fit(X_train,y_train)

MultinomialNB()

In [16]:
#prediction and checking accuracy and precision
y_pred = model_nb.predict(X_test)
accuracy = accuracy_score(y_pred,y_test)
precision = precision_score(y_pred,y_test, average = 'weighted')
precision
print("accuracy:", accuracy)
print("precision", precision)

accuracy: 0.8175321581705574
precision 0.8205654900245481


In [17]:
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92       414
           1       0.89      0.93      0.91       478
           2       0.87      0.84      0.86       420
           3       0.77      0.78      0.78       446
           4       0.78      0.72      0.75       404
           5       0.83      0.69      0.75       388
           6       0.90      0.91      0.91       404
           7       0.96      0.79      0.86       465
           8       0.66      0.76      0.71       393
           9       0.63      0.82      0.72       386

    accuracy                           0.82      4198
   macro avg       0.82      0.82      0.82      4198
weighted avg       0.83      0.82      0.82      4198



In [18]:
#cross-validation
scores = cross_val_score(model_nb, X_train, y_train, cv=3, scoring='accuracy')
print("cross validation scores", scores)

cross validation scores [0.82285714 0.81629736 0.84417441]


In [19]:
#checking confusion matrix
confusion_dt = confusion_matrix(y_test,y_pred)
confusion_dt

array([[380,   0,   0,   2,   0,   7,   6,   0,  19,   0],
       [  0, 445,   6,   5,   0,   0,   1,   0,  19,   2],
       [  3,   7, 352,   9,   6,   2,  12,   3,  25,   1],
       [  2,   8,  25, 348,   2,  15,   6,   3,  20,  17],
       [  1,   1,   3,   0, 290,   1,   5,   0,  12,  91],
       [ 11,   5,   2,  55,   5, 268,   5,   1,  25,  11],
       [  6,   0,   4,   0,   5,  14, 368,   0,   7,   0],
       [  2,   8,   1,   1,  28,   0,   1, 366,   8,  50],
       [  4,  24,   9,  23,   4,  16,   3,   1, 297,  12],
       [  0,   1,   1,   7,  32,   1,   0,   9,  17, 318]])

In [20]:
#Fine-tuning it:

param_dist = {
    'alpha': np.arange(0.01,0.1, step=0.001)
}

# Create a GridSearchCV object
randomized_search = RandomizedSearchCV(model_nb, param_distributions=param_dist, cv=3, scoring='accuracy')

# Fit the GridSearchCV object to your data
randomized_search.fit(X_train, y_train)

# Retrieve the best hyperparameters and best estimator
best_params = randomized_search.best_params_
best_model = randomized_search.best_estimator_

In [21]:
best_params

{'alpha': 0.021999999999999992}

In [22]:
#checking accuracy and prediction
y_pred = best_model.predict(X_test)
print("accuracy:", accuracy_score(y_pred,y_test))
print("precision:", precision_score(y_pred, y_test, average='weighted'))

accuracy: 0.8177703668413531
precision: 0.8205249611337472


In [23]:
#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92       414
           1       0.89      0.93      0.91       478
           2       0.87      0.84      0.86       420
           3       0.78      0.78      0.78       446
           4       0.78      0.72      0.75       404
           5       0.82      0.69      0.75       388
           6       0.91      0.91      0.91       404
           7       0.95      0.79      0.86       465
           8       0.66      0.76      0.71       393
           9       0.63      0.82      0.71       386

    accuracy                           0.82      4198
   macro avg       0.82      0.82      0.82      4198
weighted avg       0.83      0.82      0.82      4198



In [24]:
#checking confusion matrix of tuned model
confusion_dt = confusion_matrix(y_test,y_pred)
confusion_dt

array([[380,   0,   0,   2,   0,   7,   6,   0,  19,   0],
       [  0, 444,   7,   5,   0,   0,   1,   0,  19,   2],
       [  3,   7, 354,   8,   6,   2,  11,   3,  25,   1],
       [  2,   8,  25, 348,   2,  15,   6,   3,  20,  17],
       [  1,   1,   3,   0, 290,   1,   5,   0,  12,  91],
       [ 11,   5,   2,  55,   5, 268,   5,   1,  25,  11],
       [  6,   0,   5,   0,   5,  14, 367,   0,   7,   0],
       [  1,   8,   1,   1,  28,   0,   1, 367,   8,  50],
       [  4,  23,   9,  23,   4,  17,   2,   1, 298,  12],
       [  0,   1,   1,   7,  32,   1,   0,  10,  17, 317]])

In [25]:
#cross-validation
scores = cross_val_score(model_nb, X_train, y_train, cv=3, scoring='accuracy')
print("cross validation scores", scores)

cross validation scores [0.82285714 0.81629736 0.84417441]


In [26]:
#our results are almost the same after tuning the model 

[CV] END criterion=gini, max_depth=62, min_samples_leaf=9, min_samples_split=5; total time=   0.4s
[CV] END criterion=entropy, max_depth=76, min_samples_leaf=1, min_samples_split=22; total time=   0.6s
[CV] END criterion=entropy, max_depth=94, min_samples_leaf=8, min_samples_split=20; total time=   0.5s
[CV] END criterion=gini, max_depth=156, min_samples_leaf=14, min_samples_split=21; total time=   0.3s
[CV] END criterion=entropy, max_depth=20, min_samples_leaf=11, min_samples_split=25; total time=   0.5s
[CV] END criterion=gini, max_depth=12, min_samples_leaf=8, min_samples_split=3; total time=   0.3s
[CV] END criterion=gini, max_depth=12, min_samples_leaf=8, min_samples_split=3; total time=   0.3s
[CV] END criterion=entropy, max_depth=36, min_samples_leaf=6, min_samples_split=7; total time=   0.5s
[CV] END criterion=entropy, max_depth=164, min_samples_leaf=7, min_samples_split=13; total time=   0.5s
[CV] END criterion=entropy, max_depth=96, min_samples_leaf=13, min_samples_split=18; 

[CV] END criterion=gini, max_depth=12, min_samples_leaf=11, min_samples_split=22; total time=   0.3s
[CV] END criterion=entropy, max_depth=164, min_samples_leaf=21, min_samples_split=16; total time=   0.5s
[CV] END criterion=gini, max_depth=192, min_samples_leaf=12, min_samples_split=24; total time=   0.3s
[CV] END criterion=gini, max_depth=132, min_samples_leaf=25, min_samples_split=20; total time=   0.2s
[CV] END criterion=entropy, max_depth=194, min_samples_leaf=22, min_samples_split=25; total time=   0.4s
[CV] END criterion=entropy, max_depth=20, min_samples_leaf=11, min_samples_split=25; total time=   0.5s
[CV] END criterion=entropy, max_depth=36, min_samples_leaf=1, min_samples_split=22; total time=   0.5s
[CV] END criterion=entropy, max_depth=28, min_samples_leaf=24, min_samples_split=5; total time=   0.4s
[CV] END criterion=entropy, max_depth=46, min_samples_leaf=8, min_samples_split=3; total time=   0.5s
[CV] END criterion=entropy, max_depth=96, min_samples_leaf=13, min_sample

[CV] END criterion=entropy, max_depth=164, min_samples_leaf=21, min_samples_split=16; total time=   0.5s
[CV] END criterion=entropy, max_depth=76, min_samples_leaf=1, min_samples_split=22; total time=   0.5s
[CV] END criterion=gini, max_depth=132, min_samples_leaf=25, min_samples_split=20; total time=   0.3s
[CV] END criterion=entropy, max_depth=194, min_samples_leaf=22, min_samples_split=25; total time=   0.4s
[CV] END criterion=gini, max_depth=62, min_samples_leaf=18, min_samples_split=17; total time=   0.3s
[CV] END criterion=gini, max_depth=146, min_samples_leaf=13, min_samples_split=4; total time=   0.3s
[CV] END criterion=gini, max_depth=12, min_samples_leaf=8, min_samples_split=3; total time=   0.3s
[CV] END criterion=entropy, max_depth=18, min_samples_leaf=14, min_samples_split=25; total time=   0.5s
[CV] END criterion=gini, max_depth=26, min_samples_leaf=22, min_samples_split=9; total time=   0.3s
[CV] END criterion=entropy, max_depth=164, min_samples_leaf=7, min_samples_split